In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
population = pd.read_csv("/Users/katrinakonz/Desktop/OPENCLASSROOMS/P3/P3_Katrina_JUMADIAO/P3_01_qa/FAO_2013_fr/fr_population.csv")
vegetal = pd.read_csv("/Users/katrinakonz/Desktop/OPENCLASSROOMS/P3/P3_Katrina_JUMADIAO/P3_01_qa/FAO_2013_fr/fr_vegetaux.csv") 
animaux = pd.read_csv("/Users/katrinakonz/Desktop/OPENCLASSROOMS/P3/P3_Katrina_JUMADIAO/P3_01_qa/FAO_2013_fr/fr_animaux.csv")
cereals = pd.read_csv("/Users/katrinakonz/Desktop/OPENCLASSROOMS/P3/P3_Katrina_JUMADIAO/P3_01_qa/FAO_2013_fr/fr_céréales.csv")
under = pd.read_csv("/Users/katrinakonz/Desktop/OPENCLASSROOMS/P3/P3_Katrina_JUMADIAO/P3_01_qa/FAO_2013_fr/fr_sousalimentation.csv")

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
# QUESTION 1: donnez le résultat de votre calcul pour l'année 2013.

### POPULATION base

In [3]:
population.rename(columns={"Valeur": "population"}, inplace=True)
population["population"] *= 1000

total_population = population["population"].sum()
print(f"{total_population:,}")

8,413,993,000


### POPULATION sans excès (Chine)

In [4]:
population.loc[population["Zone"].str.contains("Chine", case=False)]
population.drop(index=33, inplace=True)

In [5]:
population_mondiale = population["population"].sum()
print(f"{population_mondiale:,}")

6,997,326,000


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
# Question 2 : identifiez ces redondances, en donnant votre réponse sous forme de formule mathématique.

### Disponibilité alimentaire
- import + variation + production - export
- aliment animaux + pertes + semences + traitement + nourriture + autres utilisation

### France 2013 : Disponibilité intérieure en Blé

In [6]:
vegetal.loc[(vegetal["Élément"] == "Disponibilité intérieure") & (vegetal["Produit"] == "Blé") & (vegetal["Zone"] == "France")]

france_ble = round(vegetal.iloc[37491, 11] * 1000)
print(f"{france_ble:,}")

20,298,000


### 1e equation
Disponibilité intérieure (20298) = importation (2055) + variation (1131) + production (38614) - export (21502)

In [7]:
france_ble_1 = round((vegetal.iloc[37487, 11] + vegetal.iloc[37488, 11] + vegetal.iloc[37489, 11] - vegetal.iloc[37490, 11]) * 1000)
print(f"1. Disponibilité intérieure en France (produit Blé) = {france_ble_1:,}")

1. Disponibilité intérieure en France (produit Blé) = 20,298,000


### 2e equation
Disponibilité intérieure (20298) = aliment animaux (7822) + pertes (358) + semences (748) + traitement (1575) + nourriture (6971) + autres utilisation (2824)

In [8]:
france_ble_2 = round((vegetal.iloc[37492,11] + vegetal.iloc[37494,11] + vegetal.iloc[37493,11] + vegetal.iloc[37495,11] + vegetal.iloc[37497,11] + vegetal.iloc[37496,11]) * 1000)
print(f"2. Disponibilité intérieure en France (produit Blé) = {france_ble_2:,}")

2. Disponibilité intérieure en France (produit Blé) = 20,298,000


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
# Question 3 : calculez (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal puis en kg de protéines.

In [9]:
# Concatenation: animaux + vegetaux

animaux["origine"] = "animal"
vegetal["origine"] = "vegetal"

foods = pd.concat([animaux, vegetal])

In [10]:
# Pivot: animaux + vegetaux

foods = pd.pivot_table(foods, index=["Code zone","Zone","Code Produit","Produit","Année","origine"], values="Valeur", columns="Élément")
foods = foods.rename_axis(None, axis=1)
foods.reset_index(inplace=True)


# sans excès (Chine)
foods.drop(foods[foods["Zone"] == "Chine"].index, inplace=True)

In [11]:
# Conversion des unités (milliers de tonnes)

tons_k = ["Aliments pour animaux","Autres utilisations (non alimentaire)","Disponibilité intérieure",
            "Exportations - Quantité","Importations - Quantité","Nourriture","Pertes","Production",
            "Semences","Traitement","Variation de stock"]

for list in tons_k :
    foods[list] *= 1000000

In [12]:
# Merge avec Population

pop_col = ["Zone", "population"]
pop_merge = pd.DataFrame(population[pop_col])

food_population = pd.merge(foods, pop_merge, on="Zone", how="left").copy()

# Renommer les colonnes
food_population.rename(columns={"Code zone":"country_code",
                                "Zone":"country",
                                "Code Produit":"product_code",
                                "Produit":"product",
                                "Année":"year",
                                "Aliments pour animaux":"animal_food",
                                "Autres utilisations (non alimentaire)":"other_uses",
                                "Disponibilité alimentaire (Kcal/personne/jour)":"alim_kcal/p/j",
                                "Disponibilité alimentaire en quantité (kg/personne/an)":"alim_kg/p/an",
                                "Disponibilité de matière grasse en quantité (g/personne/jour)":"matgrasse_g/p/j",
                                "Disponibilité de protéines en quantité (g/personne/jour)":"prot_g/p/j",
                                "Disponibilité intérieure":"dispo_int",
                                "Exportations - Quantité":"exports",
                                "Importations - Quantité":"imports",
                                "Nourriture":"nourriture",
                                "Pertes":"pertes",
                                "Production":"production",
                                "Semences":"semences",
                                "Traitement":"traitement",
                                "Variation de stock":"stock_var",
                                }, inplace=True)

In [13]:
# Conversions
days = 365
g_to_kg = 1000

### Disponibilité alimentaire (pour chaque pays et chaque produit) en kcal

In [14]:
food_population["alim_kcal/p/an"] = food_population["alim_kcal/p/j"] * food_population["population"] * days
food_population[["country", "alim_kcal/p/an"]].head(2)

,country,alim_kcal/p/an
0,Arménie,1.112684e+12
1,Arménie,0.000000e+00


### Disponibilité alimentaire (pour chaque pays et chaque produit) en kg de protéines

In [15]:
food_population["prot_kg/p/an"] = (food_population["prot_g/p/j"] / g_to_kg) * food_population["population"] * days
food_population[["country", "prot_kg/p/an"]].head(2)

,country,prot_kg/p/an
0,Arménie,33163184.6
1,Arménie,0.0


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
# Question 4 : calculez pour chaque produit le ratio "énergie/poids", que vous donnerez en kcal/kg.

### Ratio energie/pods en kcal/kg

In [16]:
# Remplacer les 0 par les valeurs NaN

food_population.replace(0, np.nan, inplace=True)

In [17]:
food_population["ratio_energie"] = food_population["alim_kcal/p/an"] / food_population["nourriture"]

In [18]:
# Verification avec l'Oeuf: selon Wikipedia, la valeur énergétique d'un oeuf est de 1470 kcal/kg.

# 1e methode
egg = food_population.loc[food_population["product"] == "Oeufs"].groupby("product").mean()
egg["ratio_energie"]

# 2e methode
eggs = food_population.loc[food_population["product"] == "Oeufs"]
eggs = eggs["ratio_energie"].mean()
eggs = round(eggs)
print(f"Valeur énergétique d'un oeuf est {eggs:,} kcal/kg")

Valeur énergétique d'un oeuf est 1,345 kcal/kg


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
## Transitional question
En suivant la même méthodologie, calculez également le pourcentage de protéines de chaque produit (pour chaque pays).

In [19]:
food_population["ratio_prot"] = food_population["prot_kg/p/an"] / food_population["nourriture"]

In [20]:
# Verification avec l'Avoine: selon Wikipedia, le ratio protéine de l'avoine est d'environ 10,7%.

# 1e methode
oat = food_population.loc[food_population["product"] == "Avoine"].groupby("product").mean().iloc[:]
oat["ratio_prot"]

# 2e methode
oats = food_population.loc[food_population["product"] == "Avoine"]
oats = oats["ratio_prot"].mean()
oats = round(oats * 100, 1)
print(f"Ratio protéine de l'avoine est {oats:,}%")

Ratio protéine de l'avoine est 8.4%


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
# Question 5 : citez 5 aliments parmi les 20 aliments

### Les plus caloriques

In [21]:
food_population[["product", "ratio_energie"]].groupby("product").agg("mean").sort_values(by="ratio_energie", ascending=False).head(5)

,ratio_energie
product,
Huiles de Foie de Poisso,9982.932500
Huile de Sésame,9433.744729
Huile de Son de Riz,9361.549883
Huiles de Poissons,9003.715504
Huile d'Arachide,8935.765811


### Les plus riche en protéines

In [22]:
food_population[["product", "ratio_prot"]].groupby("product").agg("mean").sort_values(by="ratio_prot", ascending=False).head(5)

,ratio_prot
product,
Graines Colza/Moutarde,0.285365
Soja,0.285331
Arachides Decortiquees,0.251466
Pois,0.223244
Légumineuses Autres,0.219716


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
# Question 6 : calculez, pour les produits végétaux uniquement, la disponibilité intérieure mondiale exprimée en kcal.

In [23]:
plant_based = food_population.loc[food_population["origine"] == "vegetal"].copy()

plant_based["dispo_int_kcal"] = plant_based["dispo_int"] * plant_based["ratio_energie"]

plant_dispo_int_kcal = round(plant_based["dispo_int_kcal"].sum(skipna=True), 2)
print(f"Disponibilité intérieure mondiale (produits végétaux): {plant_dispo_int_kcal:,} kcal")

Disponibilité intérieure mondiale (produits végétaux): 1.2312091265143276e+16 kcal


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
#  Question 7 : combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture ?

In [24]:
# Conversions
av_prot_g = 0.8
av_poids_kg = 60

prot_intake = round((av_prot_g * av_poids_kg) / g_to_kg, 2)
kcal_intake = 2200

---
### Disponibilité intérieure mondiale exprimée en (protèines) kg

In [25]:
plant_based["dispo_int_prot"] = plant_based["dispo_int"] * plant_based["ratio_prot"]

plant_dispo_int_prot = round(plant_based["dispo_int_prot"].sum(skipna=True), 2)
print(f"Disponibilité intérieure mondiale (produits végétaux): {plant_dispo_int_prot:,} kg of proteins")

Disponibilité intérieure mondiale (produits végétaux): 290,177,334,142.14 kg of proteins


---
### En termes de CALORIES et en pourcentage de la population mondiale.

In [26]:
humans_kcal = plant_based["dispo_int_kcal"].sum(skipna=True) / (kcal_intake * days)

kcal_formatted = round(humans_kcal, 2)
print(f"The calorie equivalent would feed around {kcal_formatted:,} people with the world's PLANT-based domestic supply")

# CALORIES en pourcentage
humans_kcal_percent = (kcal_formatted / population["population"].sum(skipna=True)) * 100
print(f"The calorie equivalent would feed about {humans_kcal_percent:.2f}% of the population with the world's PLANT-based domestic supply")

The calorie equivalent would feed around 15,332,616,768.55 people with the world's PLANT-based domestic supply
The calorie equivalent would feed about 219.12% of the population with the world's PLANT-based domestic supply


### En termes de PROTÈINES et en pourcentage de la population mondiale.

In [27]:
humans_prot = plant_dispo_int_prot / (prot_intake * days)

prot_formatted = round(humans_prot, 2)
print(f"The protein equivalent would feed around {prot_formatted:,} people with the world's PLANT-based domestic supply")

# PROTÉINES en pourcentage
humans_prot_percent = (prot_formatted / population["population"].sum(skipna=True)) * 100
print(f"The protein equivalent would feed about {humans_prot_percent:.2f}% of the population with the world's PLANT-based domestic supply")

The protein equivalent would feed around 15,900,127,898.2 people with the world's PLANT-based domestic supply
The protein equivalent would feed about 227.23% of the population with the world's PLANT-based domestic supply


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
#  Question 8 : combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux la nourriture végétale destinée aux animaux et les pertes de produits végétaux étaient utilisés pour de la nourriture ?

In [28]:
plant_based["plant_food_kg"] = plant_based["nourriture"] + plant_based["animal_food"] + plant_based["pertes"]

---
### Disponibilité alimentaire (nourriture, pertes et aliments pour animaux) exprimée en (calories) kcal

In [29]:
plant_based["plant_food_kcal"] = plant_based["plant_food_kg"] * plant_based["ratio_energie"]

plant_based_kcal = round(plant_based["plant_food_kcal"].sum(skipna=True))
print(f"Disponibilité alimentaire (nourriture, pertes et aliments pour animaux): {plant_based_kcal:,} kcal")

Disponibilité alimentaire (nourriture, pertes et aliments pour animaux): 5,967,656,072,762,229 kcal


### Disponibilité alimentaire (nourriture, pertes et aliments pour animaux) exprimée en (protèines) kg

In [30]:
plant_based["plant_food_prot"] = plant_based["plant_food_kg"] * plant_based["ratio_prot"]

plant_based_prot = round(plant_based["plant_food_prot"].sum(skipna=True), 2)
print(f"Disponibilité alimentaire (nourriture, pertes et aliments pour animaux): {plant_based_prot:,} kg of proteins")

Disponibilité alimentaire (nourriture, pertes et aliments pour animaux): 156,346,045,335.88 kg of proteins


---
### En termes de calories et en pourcentage de la population mondiale.

In [31]:
human_plant_kcal = plant_based_kcal / (kcal_intake * days)

human_plant_kcal = round(human_plant_kcal, 2)
print(f"The calorie equivalent would feed around {human_plant_kcal:,} of the population")

# CALORIES en pourcentage
human_plant_kcal_percent = (human_plant_kcal / population["population"].sum(skipna=True))* 100
print(f"The calorie equivalent would feed about {human_plant_kcal_percent:.2f}% of the population")

The calorie equivalent would feed around 7,431,701,211.41 of the population
The calorie equivalent would feed about 106.21% of the population


### En termes de protéines et en pourcentage de la population mondiale.

In [32]:
human_plant_prot = plant_based_prot / (prot_intake * days)

human_plant_prot = round(human_plant_prot, 2)
print(f"The protein equivalent would feed around {human_plant_prot:,} of the population")

# PROTÈINES en pourcentage
human_plant_prot_percent = (human_plant_prot / population["population"].sum(skipna=True))* 100
print(f"The protein equivalent would feed about {human_plant_prot_percent:.2f}% of the population")

The protein equivalent would feed around 8,566,906,593.75 of the population
The protein equivalent would feed about 122.43% of the population


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
# Question 9 : combien d'humains pourraient être nourris avec la disponibilité alimentaire mondiale ?

---
### Disponibilité alimentaire (vegetaux et animaux) exprimée en (calories) kcal

In [33]:
food_population["food_kcal"] = food_population["nourriture"] * food_population["ratio_energie"]

food_kcal_sum = round(food_population["food_kcal"].sum(skipna=True))
print(f"ANIMAL and PLANT based foods: {food_kcal_sum:,} kcal")

ANIMAL and PLANT based foods: 7,360,354,597,265,000 kcal


### Disponibilité alimentaire (vegetaux et animaux) exprimée en (protèines) kg

In [34]:
food_population["food_prot"] = food_population["nourriture"] * food_population["ratio_prot"]

food_prot_sum = round(food_population["food_prot"].sum(skipna=True), 2)
print(f"ANIMAL and PLANT based foods: {food_prot_sum:,} kg of proteins")

ANIMAL and PLANT based foods: 207,145,508,486.9 kg of proteins


---
### En termes de calories et en pourcentage de la population mondiale.

In [35]:
human_food_kcal = food_population["food_kcal"].sum(skipna=True) / (kcal_intake * days)

human_food_kcal = round(human_food_kcal, 2)
print(f"The calorie equivalent would feed around {human_food_kcal:,} of the population")

# CALORIES en pourcentage
human_food_kcal_percent = (human_food_kcal / population["population"].sum(skipna=True))* 100
print(f"The calorie equivalent would feed about {human_food_kcal_percent:.2f}% of the population")

The calorie equivalent would feed around 9,166,070,482.27 of the population
The calorie equivalent would feed about 130.99% of the population


### En termes de protéines et en pourcentage de la population mondiale.

In [36]:
human_food_prot = food_prot_sum / (prot_intake * days)

human_food_prot = round(human_food_prot, 2)
print(f"The protein equivalent would feed around {human_food_prot:,} of the population")

# PROTÈINES en pourcentage
human_food_prot_percent = (human_food_prot / population["population"].sum(skipna=True))* 100
print(f"The protein equivalent would feed about {human_food_prot_percent:.2f}% of the population")

The protein equivalent would feed around 11,350,438,821.2 of the population
The protein equivalent would feed about 162.21% of the population


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
# Question 10 : quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?

In [37]:
under.rename(columns= {"Code Domaine": "domain_code",
                       "Domaine": "domain",
                       "Code zone":"country_code",
                       "Zone":"country",
                       "Code Élément": "element_code",
                       "Élément": "element",
                       "Code Produit":"product_code",
                       "Produit":"product",
                       "Code année": "year_code",
                       "Année":"year",
                       "Unité": "unit",
                       "Valeur": "undernourished",
                       "Symbole": "symbol",
                       "Description du Symbole": "description",
                       "Note": "note"
                       }, inplace=True)

In [38]:
# Nettoyage des données
under["year"] = pd.to_numeric(under["year"].str[0:4])+1
under.replace('<0.1', 0.0, inplace=True)
under["undernourished"] = under["undernourished"].astype(float)

# Conversion des unités (millions)
under["undernourished"] *= 1000000

# Isolation de l'année 2013 sans excès (Chine)
under = under.loc[under["year"] == 2013]
under.loc[under["country"].str.contains("Chine", case=False)]
under.drop(index=185, inplace=True)

In [57]:
under_sum = round(under["undernourished"].sum(skipna=True))
under_percent = round(under_sum * 100 / population["population"].sum(), 2)

print(f"In 2013, there are {under_sum:,} millions of undernourished in the world, \nAccounting for {under_percent:.2f}% of the world's population")

In 2013, there are 743,700,000 millions of undernourished in the world, 
Accounting for 10.63% of the world's population


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
## Transitional question

In [40]:
cereals.rename(columns= {"Code Domaine": "domain_code",
                       "Domaine": "domain",
                       "Code zone":"country_code",
                       "Zone":"country",
                       "Code Élément": "element_code",
                       "Élément": "element",
                       "Code Produit":"product_code",
                       "Produit":"product",
                       "Code année": "year_code",
                       "Année":"year",
                       "Unité": "unit",
                       "Valeur": "values",
                       "Symbole": "symbol",
                       "Description du Symbole": "description"
                       }, inplace=True)

Établissez la liste des produits (ainsi que leur code) considéré comme des céréales selon la FAO.

In [41]:
enum_cereal = cereals[["product", "product_code"]]
enum_cereal.drop_duplicates()

,product,product_code
0,Blé,2511
1,Riz (Eq Blanchi),2805
2,Orge,2513
3,Maïs,2514
4,Millet,2517
9,Seigle,2515
10,Avoine,2516
12,Sorgho,2518
13,"Céréales, Autres",2520


Repérez dans vos données les informations concernant les céréales (par exemple en créant une colonne de type booléen nommée "is_cereal").

In [42]:
cereal_list = cereals["product"].unique()

food_population["is_cereal"] = food_population["product"].isin(cereal_list)

food_population[["country", "product", "product_code", "is_cereal"]].head(2)

,country,product,product_code,is_cereal
0,Arménie,Blé,2511,True
1,Arménie,Orge,2513,True


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
# Question 11 : quelle proportion (en termes de poids) est destinée à l'alimentation animale ?

In [43]:
only_cereal = food_population.loc[food_population["is_cereal"] == True]

only_cereal_foods = round(only_cereal["nourriture"].sum(skipna=True))
print(f"Cereals for human food = {only_cereal_foods:,}")

only_cereal_animal = round(only_cereal["animal_food"].sum(skipna=True))
print(f"Cereals for animal food = {only_cereal_animal:,}")

total_cereals = only_cereal_foods + only_cereal_animal
print(f"Total cereals = {total_cereals:,}")
print(" ")

animal_ratio_cereal = round(only_cereal_animal / (total_cereals) * 100, 2)
print(f"Cereals intended for animal feed represents {animal_ratio_cereal:.2f}% of the cereals used for human feed")

Cereals for human food = 1,029,010,000,000
Cereals for animal food = 873,535,000,000
Total cereals = 1,902,545,000,000
 
Cereals intended for animal feed represents 45.91% of the cereals used for human feed


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
## Transitional question
Repérez les 15 produits les plus exportés par ce groupe de pays.

In [44]:
#creating a list of undernourished countries from the undernourished data
under_country = under.loc[under["undernourished"] > 0]["country"].unique()

# and adding an indication to the main DataFrame
food_population["is_under"] = food_population["country"].isin(under_country)

# STEP 1 = isolating undernourished countries
unders = food_population[food_population["is_under"] == True]

# STEP 2 = returning 15 of the most exported products from them
most_exported = unders.groupby("product")["exports"].sum().sort_values(ascending=False).reset_index().head(15)
most_exported

,product,exports
0,Huile de Palme,4.628500e+10
1,Maïs,3.788800e+10
2,Manioc,3.584300e+10
3,Riz (Eq Blanchi),3.278700e+10
4,Sucre Eq Brut,2.523700e+10
5,Blé,2.484100e+10
6,"Légumes, Autres",1.994800e+10
7,Bananes,1.780500e+10
8,"Fruits, Autres",1.459000e+10
9,Lait - Excl Beurre,1.448600e+10


Parmi les données des bilans alimentaires au niveau mondial, sélectionnez les 200 plus grandes importations de ces produits (1 importation = une quantité d'un produit donné importée par un pays donné)

In [45]:
most_imported = pd.merge(most_exported, food_population, on="product", how="left").copy()

most_imported = most_imported.sort_values(by="imports", ascending=False).iloc[:200]
most_imported[["country", "product", "imports"]]

,country,product,imports
1748,"Chine, continentale",Soja,6.338100e+10
371,"Chine, continentale",Manioc,2.904600e+10
247,Japon,Maïs,1.440300e+10
900,Égypte,Blé,1.033100e+10
253,République de Corée,Maïs,8.755000e+09
...,...,...,...
605,Malaisie,Riz (Eq Blanchi),1.079000e+09
1106,Italie,"Légumes, Autres",1.061000e+09
1713,Viet Nam,Lait - Excl Beurre,1.041000e+09
981,Pologne,Blé,1.037000e+09


Groupez ces importations par produit, afin d'avoir une table contenant 1 ligne pour chacun des 15 produits.

In [46]:
most_imported = most_imported.groupby("product").sum()
# can sort values if needed

most_imported = most_imported.reset_index()
most_imported[["product", "imports"]]

,product,imports
0,Bananes,9.663000e+09
1,Blé,1.475760e+11
2,"Fruits, Autres",1.571400e+10
3,Huile de Palme,2.974700e+10
4,Lait - Excl Beurre,8.841800e+10
5,"Légumes, Autres",2.817700e+10
6,Manioc,3.465500e+10
7,Maïs,1.024510e+11
8,"Oranges, Mandarines",2.294500e+10
9,Poissons Pelagiques,1.344500e+10


Calculez pour chaque produit le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure.

In [47]:
most_imported["other_ratio"] = most_imported["other_uses"] / most_imported["dispo_int"]

most_imported[["product", "other_ratio"]]

,product,other_ratio
0,Bananes,0.000000
1,Blé,0.032963
2,"Fruits, Autres",0.000000
3,Huile de Palme,0.724803
4,Lait - Excl Beurre,0.036006
5,"Légumes, Autres",0.000000
6,Manioc,0.302335
7,Maïs,0.259771
8,"Oranges, Mandarines",0.000051
9,Poissons Pelagiques,0.000000


Calculez pour chaque produit ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)

In [48]:
most_imported["animal_food_ratio"] = most_imported["animal_food"] / most_imported["nourriture"]

most_imported[["product", "animal_food_ratio"]]

,product,animal_food_ratio
0,Bananes,0.000000
1,Blé,0.326172
2,"Fruits, Autres",0.000000
3,Huile de Palme,0.000000
4,Lait - Excl Beurre,0.090600
5,"Légumes, Autres",0.052568
6,Manioc,1.648358
7,Maïs,7.204925
8,"Oranges, Mandarines",0.000000
9,Poissons Pelagiques,3.087630


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
# Question 12 : donnez les 3 produits qui ont la plus grande valeur pour chacun des 2 ratios

In [49]:
# top 3 cereal products with the highest other uses ratio 
most_imported[["product", "other_ratio"]].sort_values(by="other_ratio", ascending=False).head(3)

,product,other_ratio
3,Huile de Palme,0.724803
6,Manioc,0.302335
7,Maïs,0.259771


In [50]:
# top 3 cereal products with the highest animal food ratio 
most_imported[["product", "animal_food_ratio"]].sort_values(by="animal_food_ratio", ascending=False).head(3)

,product,animal_food_ratio
7,Maïs,7.204925
9,Poissons Pelagiques,3.087630
12,Soja,1.825566


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
# Question 13 : combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

In [51]:
# isolating USA and cereal produces
usa = food_population.groupby("country")
usa = usa.get_group("États-Unis d'Amérique")
usa_cereal = usa[usa["is_cereal"] == True].copy()

# total production of cereals for animal feed in the USA
usa_animal = round(usa_cereal["animal_food"].sum())
print(f"Animal food production (cereals) in the USA = {usa_animal:,}")

# 10% of USA's cereal production used for animal feed
animal_portion = round(usa_animal * 0.1)
print(f"10% of the animal food production (cereals) in the USA = {animal_portion:,}")

Animal food production (cereals) in the USA = 140,096,000,000
10% of the animal food production (cereals) in the USA = 14,009,600,000


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
# Question 14 : en Thaïlande, quelle proportion de manioc est exportée ? Quelle est la proportion de personnes en sous-nutrition?

In [52]:
# total number of exported Manioc in the world
manioc_exports = food_population[food_population["product"] == "Manioc"]
manioc_exports = round(manioc_exports["exports"].sum(skipna=True))
print(f"Total Manioc products exported in the world: {manioc_exports:,}")

print(" ")

# Manoic exports in Thailand
thailand = food_population.groupby("country")
thailand = thailand.get_group("Thaïlande").copy()
manioc_thailand = thailand[thailand["product"] == "Manioc"]
manioc_thailand = round(manioc_thailand["exports"].sum())
print(f"Thailand exports a total of {manioc_thailand:,} Manioc products")

# percentage representation
manioc_thailand_proportion = round((manioc_thailand / manioc_exports) * 100,2)
print(f"Which accounts for {manioc_thailand_proportion:.2f}% of the world's total Manioc product exports")

Total Manioc products exported in the world: 36,052,000,000
 
Thailand exports a total of 25,214,000,000 Manioc products
Which accounts for 69.94% of the world's total Manioc product exports


### Verified values

In [53]:
# getting the maximum value and country of Manioc exports
manioc = food_population[food_population["product"] == "Manioc"].sort_values(by="exports")
max_manioc = round(max(manioc["exports"]))
print(f"Maximum value of exported Manioc products: {max_manioc:,}")

print(" ")

# verifying with table projection
manioc[["country", "product", "exports"]].sort_values(by="exports", ascending=False).head(3)

Maximum value of exported Manioc products: 25,214,000,000
 


,country,product,exports
13344,Thaïlande,Manioc,2.521400e+10
14724,Viet Nam,Manioc,8.973000e+09
6137,Indonésie,Manioc,8.200000e+08


In [54]:
# Thai population
thailand_population = thailand["population"].unique()
thailand_population = round(float(thailand_population[0]))
print(f"Population of Thailand: {thailand_population:,} inhabitants")

print(" ")

# undernourished in Thailand
under_thailand = under.groupby("country")
under_thailand = under_thailand.get_group("Thaïlande")
under_thailand = under_thailand["undernourished"].unique()
under_thailand = round(float(under_thailand[0]))
print(f"Numbers of undernourished people inside Thailand: {under_thailand:,}")

# percentage representation
under_thailand_proportion = (under_thailand / thailand_population) * 100
print(f"Which represents {under_thailand_proportion:.2f}% of Thailand's total populaton")

Population of Thailand: 67,011,000 inhabitants
 
Numbers of undernourished people inside Thailand: 5,600,000
Which represents 8.36% of Thailand's total populaton
